In [5]:
#All the import statements goes here
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import matplotlib as plt
import os
import pickle

path1 = "J:/SF/Thermal Load Prediction/Repos folders/loadPrediction/"
path_list = ["J:/SF/Thermal Load Prediction/Datas/Open Power System Data/opsd-household_data-2017-09-01/",
             "/Volumes/ADITHYA/Adithya/ML materials/DATA SETS/Open Power System /opsd-household_data-2017-09-01/"]
path_sd = ["/media/siddharth/New Volume/Github/TherML Data/Open Power System Data/opsd-household_data-2017-09-01/"]

if("adithya" in os.getlogin()):
    path = path_list[1]
elif("siddharth" in os.getlogin()):
    path = path_sd[0]
else:    
    path = path_list[0]

In [7]:
# read1 contains all rows and columns from csv file
read1 = pd.read_csv(path+"household_data_1min_singleindex.csv")

# read will have columns with load data
read = read1[read1.columns[2:11]]
x = np.array(read.values)

# t is to find out which minute in the data has minimum occurances of NaN
t = np.zeros(shape=(60))

for i in range(60):
    j=i;
    c=0
    while(j <= x.shape[0]-1):
        c1=0
        for k in x[j]:
            if(np.isnan(k)):
                c1+=1
        if(c1!=0):
            c+=1
        j+=60
    t[i]=c
    
# amin(t) gives minimum count and argmin(t)+1 gives minute value
print(np.amin(t))
print(np.argmin(t)+1)

1290.0
10


In [20]:
# using 1hr data
hr_data = pd.read_csv(path+"household_data_60min_singleindex.csv")
# removing the following columns
del hr_data["interpolated_values"]
del hr_data["DE_KN_residential4_grid_export"]
del hr_data["DE_KN_residential4_pv"]
del hr_data["cet_cest_timestamp"]

# yr_data will have the data from 2016-01-01 to 2017-01-01
yr_data = pd.DataFrame(hr_data.values[2137:10921])
yr_data.columns = hr_data.columns

In [22]:
# doing linear extrapolation on data ..ie.. if x is notNull and y in notNull and we have a set of nulls between them,
# we do : 
# gradient = (y-x)/(no. of nulls)
# add gradient to values from x to y recursively so that we fill the gap
df = pd.DataFrame()
for i in range(len(yr_data.columns)):
    if(i != 0):
        df = yr_data.iloc[:,i]
        flag1 = False
        for j in range(df.values.shape[0]):
            if(np.isnan(df.values[j]) and not(flag1)):
                prev = j-1
                flag1 = True
            elif(flag1 and not(np.isnan(df.values[j]))):
                grad = (df.values[j] - df.values[prev])/(j-prev)
                for k in range(prev+1, j):
                    df.values[k] = df.values[k-1] + grad
                flag1 = False
        yr_data[yr_data.columns[i]] = df.values

In [23]:
yr_data.isnull().any()

utc_timestamp                         False
DE_KN_residential4_dishwasher         False
DE_KN_residential4_ev                 False
DE_KN_residential4_freezer            False
DE_KN_residential4_grid_import        False
DE_KN_residential4_heat_pump          False
DE_KN_residential4_refrigerator       False
DE_KN_residential4_washing_machine    False
dtype: bool

In [24]:
# exporting the new non null containing data to new csv
yr_data.to_csv("Year_Data_New.csv",sep=",",encoding="utf-8")

In [28]:
hr_data.head()

,utc_timestamp,DE_KN_residential4_dishwasher,DE_KN_residential4_ev,DE_KN_residential4_freezer,DE_KN_residential4_grid_import,DE_KN_residential4_heat_pump,DE_KN_residential4_refrigerator,DE_KN_residential4_washing_machine
0,2015-10-04T01:00:00Z,0.002,0.945,0.012,0.077,0.00,0.0,0.001
1,2015-10-04T02:00:00Z,0.002,0.945,0.012,0.077,0.02,0.0,0.001
2,2015-10-04T03:00:00Z,0.002,0.945,0.012,0.077,0.03,0.0,0.001
3,2015-10-04T04:00:00Z,0.002,0.945,0.012,0.077,0.04,0.0,0.001
4,2015-10-04T05:00:00Z,0.002,0.945,0.012,0.077,0.06,0.0,0.001


In [27]:
# doing linear extrapolation on data ..ie.. if x is notNull and y in notNull and we have a set of nulls between them,
# we do : 
# gradient = (y-x)/(no. of nulls)
# add gradient to values from x to y recursively so that we fill the gap on hr data
df = pd.DataFrame()
for i in range(len(hr_data.columns)):
    if(i != 0):
        df = hr_data.iloc[:,i]
        flag1 = False
        flag2 = False
        for j in range(df.values.shape[0]):
            if(np.isnan(df.values[j]) and not(flag1)):
                if(j==0):
                    flag2 = True
                prev = j-1
                flag1 = True
            elif(flag1 and not(np.isnan(df.values[j]))):
                if(flag2):
                    flag2 = False
                    for k in range(prev+1,j):
                        df.values[k] = df.values[j]
                else:
                    grad = (df.values[j] - df.values[prev])/(j-prev)
                    for k in range(prev+1, j):
                        df.values[k] = df.values[k-1] + grad
                flag1 = False
        hr_data[hr_data.columns[i]] = df.values

In [29]:
# counted null values after interpolation in entire dataset
hr_data = hr_data.iloc[:11012,:]

nan_cnt = []

nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_dishwasher'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_ev'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_freezer'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_grid_import'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_heat_pump'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_refrigerator'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_washing_machine'])))

print(nan_cnt)

[0, 0, 0, 0, 0, 0, 0]


In [44]:
#Another efficient way to check if there are any NULLS present in any of the columns.
hr_data.isnull().sum(axis=0)

utc_timestamp                         0
DE_KN_residential4_dishwasher         0
DE_KN_residential4_ev                 0
DE_KN_residential4_freezer            0
DE_KN_residential4_grid_import        0
DE_KN_residential4_heat_pump          0
DE_KN_residential4_refrigerator       0
DE_KN_residential4_washing_machine    0
dtype: int64

In [39]:
hr_data.to_csv("Hour_Data_New.csv",sep=",",encoding="utf-8")
#hr_data = pd.read_csv(path1+"Hour_Data_New.csv")
#del hr_data['Unnamed: 0']

In [49]:
# relu activation for input layer alone.. output layer softmax made it worser..
# this is on entire dataset till first row with null values after interpolation
acc_dict = {}

for counter in xrange(5):
    for i in range(len(hr_data.columns)):
        df = pd.DataFrame()
        if(i != 0):
            print(hr_data.columns[i])
            df = hr_data[hr_data.columns[i]]
            ndf = pd.concat([df.shift(2), df.shift(1), df], axis=1)
            ndf.columns = ('t-2', 't-1', 't')
            ndf = ndf.iloc[2:]

            x = ndf[ndf.columns[0:2]]
            y = ndf[ndf.columns[-1:]]

            X_train,X_test,y_train,y_test = train_test_split(x.values,y.values,test_size=0.2,random_state=1)
            model = Sequential()
            model.add(Dense(5,input_dim=2,kernel_initializer='normal', activation='relu'))
            #Output Layer.
            model.add(Dense(1,kernel_initializer='normal'))
            #Compile model.
            model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mean_squared_error'])
            #Fit the model.
            model.fit(X_train,y_train,epochs=300,batch_size=200,verbose=2)
            #Final Evaluation of the model.
            scores = model.evaluate(X_test,y_test,verbose=0)
            print ("Baseline Error : %.5f" % (100-scores[1]*100))
            acc_dict.setdefault(counter,[]).append((100-scores[1]*100))
            model.summary()

DE_KN_residential4_dishwasher
Epoch 1/300
0s - loss: 4609.0851 - mean_squared_error: 4609.0851
Epoch 2/300
0s - loss: 4163.6214 - mean_squared_error: 4163.6214
Epoch 3/300
0s - loss: 3297.0860 - mean_squared_error: 3297.0860
Epoch 4/300
0s - loss: 2203.3549 - mean_squared_error: 2203.3549
Epoch 5/300
0s - loss: 1184.7126 - mean_squared_error: 1184.7126
Epoch 6/300
0s - loss: 486.1976 - mean_squared_error: 486.1976
Epoch 7/300
0s - loss: 142.6667 - mean_squared_error: 142.6667
Epoch 8/300
0s - loss: 28.8401 - mean_squared_error: 28.8401
Epoch 9/300
0s - loss: 4.1042 - mean_squared_error: 4.1042
Epoch 10/300
0s - loss: 0.5156 - mean_squared_error: 0.5156
Epoch 11/300
0s - loss: 0.1392 - mean_squared_error: 0.1392
Epoch 12/300
0s - loss: 0.1115 - mean_squared_error: 0.1115
Epoch 13/300
0s - loss: 0.1097 - mean_squared_error: 0.1097
Epoch 14/300
0s - loss: 0.1093 - mean_squared_error: 0.1093
Epoch 15/300
0s - loss: 0.1090 - mean_squared_error: 0.1090
Epoch 16/300
0s - loss: 0.1087 - mean_s

0s - loss: 0.0131 - mean_squared_error: 0.0131
Epoch 139/300
0s - loss: 0.0127 - mean_squared_error: 0.0127
Epoch 140/300
0s - loss: 0.0123 - mean_squared_error: 0.0123
Epoch 141/300
0s - loss: 0.0119 - mean_squared_error: 0.0119
Epoch 142/300
0s - loss: 0.0116 - mean_squared_error: 0.0116
Epoch 143/300
0s - loss: 0.0113 - mean_squared_error: 0.0113
Epoch 144/300
0s - loss: 0.0110 - mean_squared_error: 0.0110
Epoch 145/300
0s - loss: 0.0108 - mean_squared_error: 0.0108
Epoch 146/300
0s - loss: 0.0105 - mean_squared_error: 0.0105
Epoch 147/300
0s - loss: 0.0102 - mean_squared_error: 0.0102
Epoch 148/300
0s - loss: 0.0100 - mean_squared_error: 0.0100
Epoch 149/300
0s - loss: 0.0098 - mean_squared_error: 0.0098
Epoch 150/300
0s - loss: 0.0095 - mean_squared_error: 0.0095
Epoch 151/300
0s - loss: 0.0093 - mean_squared_error: 0.0093
Epoch 152/300
0s - loss: 0.0091 - mean_squared_error: 0.0091
Epoch 153/300
0s - loss: 0.0090 - mean_squared_error: 0.0090
Epoch 154/300
0s - loss: 0.0088 - mean

Epoch 276/300
0s - loss: 0.0076 - mean_squared_error: 0.0076
Epoch 277/300
0s - loss: 0.0081 - mean_squared_error: 0.0081
Epoch 278/300
0s - loss: 0.0074 - mean_squared_error: 0.0074
Epoch 279/300
0s - loss: 0.0081 - mean_squared_error: 0.0081
Epoch 280/300
0s - loss: 0.0077 - mean_squared_error: 0.0077
Epoch 281/300
0s - loss: 0.0080 - mean_squared_error: 0.0080
Epoch 282/300
0s - loss: 0.0077 - mean_squared_error: 0.0077
Epoch 283/300
0s - loss: 0.0078 - mean_squared_error: 0.0078
Epoch 284/300
0s - loss: 0.0080 - mean_squared_error: 0.0080
Epoch 285/300
0s - loss: 0.0077 - mean_squared_error: 0.0077
Epoch 286/300
0s - loss: 0.0076 - mean_squared_error: 0.0076
Epoch 287/300
0s - loss: 0.0085 - mean_squared_error: 0.0085
Epoch 288/300
0s - loss: 0.0075 - mean_squared_error: 0.0075
Epoch 289/300
0s - loss: 0.0162 - mean_squared_error: 0.0162
Epoch 290/300
0s - loss: 0.0078 - mean_squared_error: 0.0078
Epoch 291/300
0s - loss: 0.0075 - mean_squared_error: 0.0075
Epoch 292/300
0s - loss:

0s - loss: 0.4780 - mean_squared_error: 0.4780
Epoch 101/300
0s - loss: 0.4781 - mean_squared_error: 0.4781
Epoch 102/300
0s - loss: 0.4776 - mean_squared_error: 0.4776
Epoch 103/300
0s - loss: 0.4773 - mean_squared_error: 0.4773
Epoch 104/300
0s - loss: 0.4770 - mean_squared_error: 0.4770
Epoch 105/300
0s - loss: 0.4772 - mean_squared_error: 0.4772
Epoch 106/300
0s - loss: 0.4766 - mean_squared_error: 0.4766
Epoch 107/300
0s - loss: 0.4762 - mean_squared_error: 0.4762
Epoch 108/300
0s - loss: 0.4757 - mean_squared_error: 0.4757
Epoch 109/300
0s - loss: 0.4759 - mean_squared_error: 0.4759
Epoch 110/300
0s - loss: 0.4751 - mean_squared_error: 0.4751
Epoch 111/300
0s - loss: 0.4751 - mean_squared_error: 0.4751
Epoch 112/300
0s - loss: 0.4746 - mean_squared_error: 0.4746
Epoch 113/300
0s - loss: 0.4745 - mean_squared_error: 0.4745
Epoch 114/300
0s - loss: 0.4753 - mean_squared_error: 0.4753
Epoch 115/300
0s - loss: 0.4737 - mean_squared_error: 0.4737
Epoch 116/300
0s - loss: 0.4733 - mean

0s - loss: 0.3670 - mean_squared_error: 0.3670
Epoch 236/300
0s - loss: 0.3659 - mean_squared_error: 0.3659
Epoch 237/300
0s - loss: 0.3721 - mean_squared_error: 0.3721
Epoch 238/300
0s - loss: 0.3640 - mean_squared_error: 0.3640
Epoch 239/300
0s - loss: 0.3630 - mean_squared_error: 0.3630
Epoch 240/300
0s - loss: 0.3668 - mean_squared_error: 0.3668
Epoch 241/300
0s - loss: 0.3629 - mean_squared_error: 0.3629
Epoch 242/300
0s - loss: 0.3628 - mean_squared_error: 0.3628
Epoch 243/300
0s - loss: 0.3622 - mean_squared_error: 0.3622
Epoch 244/300
0s - loss: 0.3571 - mean_squared_error: 0.3571
Epoch 245/300
0s - loss: 0.3596 - mean_squared_error: 0.3596
Epoch 246/300
0s - loss: 0.3564 - mean_squared_error: 0.3564
Epoch 247/300
0s - loss: 0.3592 - mean_squared_error: 0.3592
Epoch 248/300
0s - loss: 0.3560 - mean_squared_error: 0.3560
Epoch 249/300
0s - loss: 0.3542 - mean_squared_error: 0.3542
Epoch 250/300
0s - loss: 0.3547 - mean_squared_error: 0.3547
Epoch 251/300
0s - loss: 0.3556 - mean

...

0s - loss: 0.2105 - mean_squared_error: 0.2105
Epoch 39/300
0s - loss: 0.2099 - mean_squared_error: 0.2099
Epoch 40/300
0s - loss: 0.2092 - mean_squared_error: 0.2092
Epoch 41/300
0s - loss: 0.2085 - mean_squared_error: 0.2085
Epoch 42/300
0s - loss: 0.2079 - mean_squared_error: 0.2079
Epoch 43/300
0s - loss: 0.2072 - mean_squared_error: 0.2072
Epoch 44/300
0s - loss: 0.2065 - mean_squared_error: 0.2065
Epoch 45/300
0s - loss: 0.2057 - mean_squared_error: 0.2057
Epoch 46/300
0s - loss: 0.2050 - mean_squared_error: 0.2050
Epoch 47/300
0s - loss: 0.2042 - mean_squared_error: 0.2042
Epoch 48/300
0s - loss: 0.2034 - mean_squared_error: 0.2034
Epoch 49/300
0s - loss: 0.2026 - mean_squared_error: 0.2026
Epoch 50/300
0s - loss: 0.2018 - mean_squared_error: 0.2018
Epoch 51/300
0s - loss: 0.2010 - mean_squared_error: 0.2010
Epoch 52/300
0s - loss: 0.2001 - mean_squared_error: 0.2001
Epoch 53/300
0s - loss: 0.1993 - mean_squared_error: 0.1993
Epoch 54/300
0s - loss: 0.1984 - mean_squared_error: 

0s - loss: 0.0096 - mean_squared_error: 0.0096
Epoch 177/300
0s - loss: 0.0089 - mean_squared_error: 0.0089
Epoch 178/300
0s - loss: 0.0083 - mean_squared_error: 0.0083
Epoch 179/300
0s - loss: 0.0077 - mean_squared_error: 0.0077
Epoch 180/300
0s - loss: 0.0071 - mean_squared_error: 0.0071
Epoch 181/300
0s - loss: 0.0065 - mean_squared_error: 0.0065
Epoch 182/300
0s - loss: 0.0060 - mean_squared_error: 0.0060
Epoch 183/300
0s - loss: 0.0056 - mean_squared_error: 0.0056
Epoch 184/300
0s - loss: 0.0051 - mean_squared_error: 0.0051
Epoch 185/300
0s - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 186/300
0s - loss: 0.0043 - mean_squared_error: 0.0043
Epoch 187/300
0s - loss: 0.0039 - mean_squared_error: 0.0039
Epoch 188/300
0s - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 189/300
0s - loss: 0.0033 - mean_squared_error: 0.0033
Epoch 190/300
0s - loss: 0.0030 - mean_squared_error: 0.0030
Epoch 191/300
0s - loss: 0.0027 - mean_squared_error: 0.0027
Epoch 192/300
0s - loss: 0.0025 - mean

0s - loss: 2.2515e-04 - mean_squared_error: 2.2515e-04
Epoch 300/300
0s - loss: 2.3511e-04 - mean_squared_error: 2.3511e-04
Baseline Error : 99.97628
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_103 (Dense)            (None, 5)                 15        
_________________________________________________________________
dense_104 (Dense)            (None, 1)                 6         
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________
DE_KN_residential4_washing_machine
Epoch 1/300
0s - loss: 5799.3872 - mean_squared_error: 5799.3872
Epoch 2/300
0s - loss: 5133.1574 - mean_squared_error: 5133.1574
Epoch 3/300
0s - loss: 3846.7171 - mean_squared_error: 3846.7171
Epoch 4/300
0s - loss: 2229.6752 - mean_squared_error: 2229.6752
Epoch 5/300
0s - loss: 909.3678 - mean_squared_error: 909.3678
Epoch 6/300
0s - loss: 235.7069 - 

0s - loss: 0.0145 - mean_squared_error: 0.0145
Epoch 127/300
0s - loss: 0.0142 - mean_squared_error: 0.0142
Epoch 128/300
0s - loss: 0.0138 - mean_squared_error: 0.0138
Epoch 129/300
0s - loss: 0.0135 - mean_squared_error: 0.0135
Epoch 130/300
0s - loss: 0.0132 - mean_squared_error: 0.0132
Epoch 131/300
0s - loss: 0.0129 - mean_squared_error: 0.0129
Epoch 132/300
0s - loss: 0.0126 - mean_squared_error: 0.0126
Epoch 133/300
0s - loss: 0.0124 - mean_squared_error: 0.0124
Epoch 134/300
0s - loss: 0.0122 - mean_squared_error: 0.0122
Epoch 135/300
0s - loss: 0.0120 - mean_squared_error: 0.0120
Epoch 136/300
0s - loss: 0.0117 - mean_squared_error: 0.0117
Epoch 137/300
0s - loss: 0.0119 - mean_squared_error: 0.0119
Epoch 138/300
0s - loss: 0.0113 - mean_squared_error: 0.0113
Epoch 139/300
0s - loss: 0.0112 - mean_squared_error: 0.0112
Epoch 140/300
0s - loss: 0.0110 - mean_squared_error: 0.0110
Epoch 141/300
0s - loss: 0.0108 - mean_squared_error: 0.0108
Epoch 142/300
0s - loss: 0.0107 - mean

0s - loss: 0.0099 - mean_squared_error: 0.0099
Epoch 262/300
0s - loss: 0.0112 - mean_squared_error: 0.0112
Epoch 263/300
0s - loss: 0.0097 - mean_squared_error: 0.0097
Epoch 264/300
0s - loss: 0.0113 - mean_squared_error: 0.0113
Epoch 265/300
0s - loss: 0.0099 - mean_squared_error: 0.0099
Epoch 266/300
0s - loss: 0.0099 - mean_squared_error: 0.0099
Epoch 267/300
0s - loss: 0.0096 - mean_squared_error: 0.0096
Epoch 268/300
0s - loss: 0.0099 - mean_squared_error: 0.0099
Epoch 269/300
0s - loss: 0.0098 - mean_squared_error: 0.0098
Epoch 270/300
0s - loss: 0.0101 - mean_squared_error: 0.0101
Epoch 271/300
0s - loss: 0.0097 - mean_squared_error: 0.0097
Epoch 272/300
0s - loss: 0.0102 - mean_squared_error: 0.0102
Epoch 273/300
0s - loss: 0.0103 - mean_squared_error: 0.0103
Epoch 274/300
0s - loss: 0.0109 - mean_squared_error: 0.0109
Epoch 275/300
0s - loss: 0.0105 - mean_squared_error: 0.0105
Epoch 276/300
0s - loss: 0.0097 - mean_squared_error: 0.0097
Epoch 277/300
0s - loss: 0.0097 - mean

In [54]:
#pickle.dump(acc_dict,open('acc_dict','wb'))
acc_dict = pickle.load(open('acc_dict','rb'))

In [56]:
'''
'Dishwasher' and 'Washing Machine' seem to be alright. Their accuracy remains similar.
But for the other attributes, it becomes negative at times. Why though? It should be something wrong with the way 
the data was interpolated(not using the graphical method) ?
'''
acc_dict

{0: [99.160624300870921,
  66.351393663699142,
  -1082488.7570248637,
  -40.475386170015668,
  51.007268550068979,
  99.97438543803311,
  99.183682942226682],
 1: [99.218887525078145,
  66.444416626776956,
  99.987170955871321,
  21.997674668950452,
  63.311279163806681,
  -899509.36595282692,
  99.126005713611363],
 2: [99.196965893899488,
  49.184226721768809,
  99.984742511079091,
  37.882776925176195,
  71.359430815175273,
  99.970936231610878,
  99.120346521962944],
 3: [99.232391909354561,
  68.965490871903356,
  99.992277716056449,
  -48.704974138119553,
  -16.361948893786987,
  99.94726629345044,
  99.160877056536165],
 4: [99.236996726531132,
  68.542564909509309,
  99.991007056799646,
  40.212594672401423,
  66.573431882178312,
  99.976283093380104,
  99.01197878491611]}

In [64]:
# relu activation for input layer alone.. output layer softmax made it worser..
# this is on the year 2016-01-01 to 2017-01-01 data
acc_dict1={}

for counter in xrange(5):
    for i in range(len(yr_data.columns)):
        df = pd.DataFrame()
        if(i != 0):
            print(yr_data.columns[i])
            df = yr_data[yr_data.columns[i]]
            ndf = pd.concat([df.shift(2), df.shift(1), df], axis=1)
            ndf.columns = ('t-2', 't-1', 't')
            ndf = ndf.iloc[2:]

            x = ndf[ndf.columns[0:2]]
            y = ndf[ndf.columns[-1:]]

            X_train,X_test,y_train,y_test = train_test_split(x.values,y.values,test_size=0.2,random_state=1)
            model = Sequential()
            model.add(Dense(5,input_dim=2,kernel_initializer='normal', activation='relu'))
            #Output Layer.
            model.add(Dense(1,kernel_initializer='normal'))
            #Compile model.
            model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mean_squared_error'])
            #Fit the model.
            model.fit(X_train,y_train,epochs=200,batch_size=200,verbose=2)
            #Final Evaluation of the model.
            scores = model.evaluate(X_test,y_test,verbose=0)
            print ("Baseline Error : %.5f" % (100-scores[1]*100))
            acc_dict1.setdefault(counter,[]).append((100-scores[1]*100))
            model.summary()

DE_KN_residential4_dishwasher
Epoch 1/200
0s - loss: 5724.0239 - mean_squared_error: 5724.0239
Epoch 2/200
0s - loss: 5505.1991 - mean_squared_error: 5505.1991
Epoch 3/200
0s - loss: 4905.0994 - mean_squared_error: 4905.0994
Epoch 4/200
0s - loss: 3935.2756 - mean_squared_error: 3935.2756
Epoch 5/200
0s - loss: 2784.7986 - mean_squared_error: 2784.7986
Epoch 6/200
0s - loss: 1694.6641 - mean_squared_error: 1694.6641
Epoch 7/200
0s - loss: 857.5615 - mean_squared_error: 857.5615
Epoch 8/200
0s - loss: 347.7844 - mean_squared_error: 347.7844
Epoch 9/200
0s - loss: 110.3588 - mean_squared_error: 110.3588
Epoch 10/200
0s - loss: 26.9582 - mean_squared_error: 26.9582
Epoch 11/200
0s - loss: 5.1313 - mean_squared_error: 5.1313
Epoch 12/200
0s - loss: 0.8122 - mean_squared_error: 0.8122
Epoch 13/200
0s - loss: 0.1444 - mean_squared_error: 0.1444
Epoch 14/200
0s - loss: 0.0637 - mean_squared_error: 0.0637
Epoch 15/200
0s - loss: 0.0555 - mean_squared_error: 0.0555
Epoch 16/200
0s - loss: 0.054

Epoch 137/200
0s - loss: 0.0354 - mean_squared_error: 0.0354
Epoch 138/200
0s - loss: 0.0351 - mean_squared_error: 0.0351
Epoch 139/200
0s - loss: 0.0348 - mean_squared_error: 0.0348
Epoch 140/200
0s - loss: 0.0345 - mean_squared_error: 0.0345
Epoch 141/200
0s - loss: 0.0342 - mean_squared_error: 0.0342
Epoch 142/200
0s - loss: 0.0339 - mean_squared_error: 0.0339
Epoch 143/200
0s - loss: 0.0336 - mean_squared_error: 0.0336
Epoch 144/200
0s - loss: 0.0332 - mean_squared_error: 0.0332
Epoch 145/200
0s - loss: 0.0329 - mean_squared_error: 0.0329
Epoch 146/200
0s - loss: 0.0326 - mean_squared_error: 0.0326
Epoch 147/200
0s - loss: 0.0323 - mean_squared_error: 0.0323
Epoch 148/200
0s - loss: 0.0319 - mean_squared_error: 0.0319
Epoch 149/200
0s - loss: 0.0316 - mean_squared_error: 0.0316
Epoch 150/200
0s - loss: 0.0313 - mean_squared_error: 0.0313
Epoch 151/200
0s - loss: 0.0309 - mean_squared_error: 0.0309
Epoch 152/200
0s - loss: 0.0306 - mean_squared_error: 0.0306
Epoch 153/200
0s - loss:

0s - loss: 0.3940 - mean_squared_error: 0.3940
Epoch 62/200
0s - loss: 0.3940 - mean_squared_error: 0.3940
Epoch 63/200
0s - loss: 0.3939 - mean_squared_error: 0.3939
Epoch 64/200
0s - loss: 0.3940 - mean_squared_error: 0.3940
Epoch 65/200
0s - loss: 0.3940 - mean_squared_error: 0.3940
Epoch 66/200
0s - loss: 0.3941 - mean_squared_error: 0.3941
Epoch 67/200
0s - loss: 0.3943 - mean_squared_error: 0.3943
Epoch 68/200
0s - loss: 0.3939 - mean_squared_error: 0.3939
Epoch 69/200
0s - loss: 0.3938 - mean_squared_error: 0.3938
Epoch 70/200
0s - loss: 0.3939 - mean_squared_error: 0.3939
Epoch 71/200
0s - loss: 0.3937 - mean_squared_error: 0.3937
Epoch 72/200
0s - loss: 0.3942 - mean_squared_error: 0.3942
Epoch 73/200
0s - loss: 0.3937 - mean_squared_error: 0.3937
Epoch 74/200
0s - loss: 0.3937 - mean_squared_error: 0.3937
Epoch 75/200
0s - loss: 0.3937 - mean_squared_error: 0.3937
Epoch 76/200
0s - loss: 0.3938 - mean_squared_error: 0.3938
Epoch 77/200
0s - loss: 0.3938 - mean_squared_error: 

0s - loss: 0.3845 - mean_squared_error: 0.3845
Epoch 198/200
0s - loss: 0.3843 - mean_squared_error: 0.3843
Epoch 199/200
0s - loss: 0.3843 - mean_squared_error: 0.3843
Epoch 200/200
0s - loss: 0.3835 - mean_squared_error: 0.3835
Baseline Error : 62.70151
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_137 (Dense)            (None, 5)                 15        
_________________________________________________________________
dense_138 (Dense)            (None, 1)                 6         
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________
DE_KN_residential4_freezer
Epoch 1/200
0s - loss: 16127.9665 - mean_squared_error: 16127.9665
Epoch 2/200
0s - loss: 15096.2122 - mean_squared_error: 15096.2122
Epoch 3/200
0s - loss: 13656.4172 - mean_squared_error: 13656.4172
Epoch 4/200
0s - loss: 11661.0768 - mean_squared_error: 11

0s - loss: 0.0593 - mean_squared_error: 0.0593
Epoch 123/200
0s - loss: 0.0590 - mean_squared_error: 0.0590
Epoch 124/200
0s - loss: 0.0586 - mean_squared_error: 0.0586
Epoch 125/200
0s - loss: 0.0582 - mean_squared_error: 0.0582
Epoch 126/200
0s - loss: 0.0579 - mean_squared_error: 0.0579
Epoch 127/200
0s - loss: 0.0575 - mean_squared_error: 0.0575
Epoch 128/200
0s - loss: 0.0571 - mean_squared_error: 0.0571
Epoch 129/200
0s - loss: 0.0567 - mean_squared_error: 0.0567
Epoch 130/200
0s - loss: 0.0563 - mean_squared_error: 0.0563
Epoch 131/200
0s - loss: 0.0560 - mean_squared_error: 0.0560
Epoch 132/200
0s - loss: 0.0555 - mean_squared_error: 0.0555
Epoch 133/200
0s - loss: 0.0552 - mean_squared_error: 0.0552
Epoch 134/200
0s - loss: 0.0547 - mean_squared_error: 0.0547
Epoch 135/200
0s - loss: 0.0543 - mean_squared_error: 0.0543
Epoch 136/200
0s - loss: 0.0539 - mean_squared_error: 0.0539
Epoch 137/200
0s - loss: 0.0535 - mean_squared_error: 0.0535
Epoch 138/200
0s - loss: 0.0530 - mean

...

0s - loss: 0.4915 - mean_squared_error: 0.4915
Epoch 199/200
0s - loss: 0.4922 - mean_squared_error: 0.4922
Epoch 200/200
0s - loss: 0.4952 - mean_squared_error: 0.4952
Baseline Error : 48.91138
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_197 (Dense)            (None, 5)                 15        
_________________________________________________________________
dense_198 (Dense)            (None, 1)                 6         
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________
DE_KN_residential4_heat_pump
Epoch 1/200
0s - loss: 4649068.9644 - mean_squared_error: 4649068.9644
Epoch 2/200
0s - loss: 4356378.8541 - mean_squared_error: 4356378.8541
Epoch 3/200
0s - loss: 3909356.8559 - mean_squared_error: 3909356.8559
Epoch 4/200
0s - loss: 3319041.2153 - mean_squared_error: 3319041.2153
Epoch 5/200
0s - loss: 2636770.398

0s - loss: 0.2897 - mean_squared_error: 0.2897
Epoch 125/200
0s - loss: 0.2897 - mean_squared_error: 0.2897
Epoch 126/200
0s - loss: 0.2898 - mean_squared_error: 0.2898
Epoch 127/200
0s - loss: 0.2897 - mean_squared_error: 0.2897
Epoch 128/200
0s - loss: 0.2897 - mean_squared_error: 0.2897
Epoch 129/200
0s - loss: 0.2895 - mean_squared_error: 0.2895
Epoch 130/200
0s - loss: 0.2900 - mean_squared_error: 0.2900
Epoch 131/200
0s - loss: 0.2897 - mean_squared_error: 0.2897
Epoch 132/200
0s - loss: 0.2897 - mean_squared_error: 0.2897
Epoch 133/200
0s - loss: 0.2897 - mean_squared_error: 0.2897
Epoch 134/200
0s - loss: 0.2897 - mean_squared_error: 0.2897
Epoch 135/200
0s - loss: 0.2897 - mean_squared_error: 0.2897
Epoch 136/200
0s - loss: 0.2897 - mean_squared_error: 0.2897
Epoch 137/200
0s - loss: 0.2900 - mean_squared_error: 0.2900
Epoch 138/200
0s - loss: 0.2896 - mean_squared_error: 0.2896
Epoch 139/200
0s - loss: 0.2896 - mean_squared_error: 0.2896
Epoch 140/200
0s - loss: 0.2899 - mean

0s - loss: 0.0740 - mean_squared_error: 0.0740
Epoch 50/200
0s - loss: 0.0738 - mean_squared_error: 0.0738
Epoch 51/200
0s - loss: 0.0736 - mean_squared_error: 0.0736
Epoch 52/200
0s - loss: 0.0734 - mean_squared_error: 0.0734
Epoch 53/200
0s - loss: 0.0732 - mean_squared_error: 0.0732
Epoch 54/200
0s - loss: 0.0730 - mean_squared_error: 0.0730
Epoch 55/200
0s - loss: 0.0728 - mean_squared_error: 0.0728
Epoch 56/200
0s - loss: 0.0726 - mean_squared_error: 0.0726
Epoch 57/200
0s - loss: 0.0724 - mean_squared_error: 0.0724
Epoch 58/200
0s - loss: 0.0722 - mean_squared_error: 0.0722
Epoch 59/200
0s - loss: 0.0720 - mean_squared_error: 0.0720
Epoch 60/200
0s - loss: 0.0718 - mean_squared_error: 0.0718
Epoch 61/200
0s - loss: 0.0715 - mean_squared_error: 0.0715
Epoch 62/200
0s - loss: 0.0713 - mean_squared_error: 0.0713
Epoch 63/200
0s - loss: 0.0711 - mean_squared_error: 0.0711
Epoch 64/200
0s - loss: 0.0708 - mean_squared_error: 0.0708
Epoch 65/200
0s - loss: 0.0706 - mean_squared_error: 

Epoch 187/200
0s - loss: 0.0146 - mean_squared_error: 0.0146
Epoch 188/200
0s - loss: 0.0141 - mean_squared_error: 0.0141
Epoch 189/200
0s - loss: 0.0136 - mean_squared_error: 0.0136
Epoch 190/200
0s - loss: 0.0132 - mean_squared_error: 0.0132
Epoch 191/200
0s - loss: 0.0128 - mean_squared_error: 0.0128
Epoch 192/200
0s - loss: 0.0123 - mean_squared_error: 0.0123
Epoch 193/200
0s - loss: 0.0119 - mean_squared_error: 0.0119
Epoch 194/200
0s - loss: 0.0115 - mean_squared_error: 0.0115
Epoch 195/200
0s - loss: 0.0111 - mean_squared_error: 0.0111
Epoch 196/200
0s - loss: 0.0107 - mean_squared_error: 0.0107
Epoch 197/200
0s - loss: 0.0103 - mean_squared_error: 0.0103
Epoch 198/200
0s - loss: 0.0099 - mean_squared_error: 0.0099
Epoch 199/200
0s - loss: 0.0095 - mean_squared_error: 0.0095
Epoch 200/200
0s - loss: 0.0092 - mean_squared_error: 0.0092
Baseline Error : 99.13713
_________________________________________________________________
Layer (type)                 Output Shape             

0s - loss: 0.1011 - mean_squared_error: 0.1011
Epoch 112/200
0s - loss: 0.1007 - mean_squared_error: 0.1007
Epoch 113/200
0s - loss: 0.1002 - mean_squared_error: 0.1002
Epoch 114/200
0s - loss: 0.0998 - mean_squared_error: 0.0998
Epoch 115/200
0s - loss: 0.0993 - mean_squared_error: 0.0993
Epoch 116/200
0s - loss: 0.0988 - mean_squared_error: 0.0988
Epoch 117/200
0s - loss: 0.0983 - mean_squared_error: 0.0983
Epoch 118/200
0s - loss: 0.0979 - mean_squared_error: 0.0979
Epoch 119/200
0s - loss: 0.0974 - mean_squared_error: 0.0974
Epoch 120/200
0s - loss: 0.0969 - mean_squared_error: 0.0969
Epoch 121/200
0s - loss: 0.0964 - mean_squared_error: 0.0964
Epoch 122/200
0s - loss: 0.0959 - mean_squared_error: 0.0959
Epoch 123/200
0s - loss: 0.0954 - mean_squared_error: 0.0954
Epoch 124/200
0s - loss: 0.0949 - mean_squared_error: 0.0949
Epoch 125/200
0s - loss: 0.0943 - mean_squared_error: 0.0943
Epoch 126/200
0s - loss: 0.0938 - mean_squared_error: 0.0938
Epoch 127/200
0s - loss: 0.0933 - mean

In [67]:
#pickle.dump(acc_dict1,open('acc_dict1','wb'))
acc_dict1 = pickle.load(open('acc_dict1','rb'))

In [68]:
acc_dict1

{0: [98.717805797071975,
  62.701512975793165,
  97.988387861320362,
  48.211459188754411,
  -485072510.67159933,
  98.361988383513335,
  98.381902832022661],
 1: [98.296165366020475,
  62.733447463524634,
  95.624056314931849,
  46.690713952194244,
  70.346646754661606,
  98.484823897779918,
  96.289621099117269],
 2: [98.799184559673222,
  59.957366922258586,
  98.824171050291085,
  47.849905168596287,
  70.398386954989832,
  99.511704499041898,
  96.010050556888004],
 3: [-480217.04307213111,
  61.041940420475889,
  98.839693204603137,
  48.859955590084724,
  68.924716673049318,
  98.340648546439638,
  -633193.00123723852],
 4: [99.035580515942087,
  60.288010892979315,
  97.944072893501072,
  48.91138439430862,
  70.890445207491069,
  99.137134069770866,
  96.042668884095306]}

In [ ]:
# plotting the dishwasher load for every 24hrs
i = 0
while(i <= x.shape[0]):
    s = np.array(read1.values[i:i+1439,0])
    k = [t[-9:-7]+t[-6:-4] for t in s ]
    k = [int(p) for p in k]
    l = read.values[i:i+1439,0]
    plt.grid(True)
    plt.plot(k,l)
    plt.show()
    i+=1440

In [82]:
d60 = pd.read_csv(path+"household_data_60min_singleindex.csv")
del d60['interpolated_values']
d60_cont1 = pd.DataFrame(d60.values[4893:8275])
print d60_cont1.shape

(3382, 11)


In [83]:
#Now try feeding this to the nn and see the results. This will help us in debugging what went wrong.
d60_cont1.isnull().sum(axis=0)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
dtype: int64